In [24]:
# Set up
%pylab
%pylab inline
%matplotlib inline
%load_ext autoreload
%autoreload 2
import tqdm
import random
import pandas as pd
from collections import Counter
from itertools import cycle

from sklearn import datasets, metrics, tree
from sklearn.cluster import KMeans, MiniBatchKMeans, Birch, DBSCAN
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

import spectral
import seaborn as sns 
import tqdm

import DecisionTree

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib
Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Zadanie 2. (1 punkt)
W pakiecie SciKit do Pythona dostępnych jest kilka popularnych algorytmów klasyfikacji danych <br/>
przy użyciu drzew klasyfikacyjnych. Zapoznaj się z nimi wykonując skrypt umieszczony w<br/>
materiałach do wykładu. Do rysowania drzew użyj programu Graphviz.<br/>
<br/>
a) Jaką miarę różnorodności stosuje algorytm konstrukcji drzew klasyfikujących użyty w skrypcie?<br/>
Dla danych Titanic sporządź drzewa klasyfikacyjne stosując indeks Giniego oraz entropię.<br/>
<br/>
b) Podziel dane Titanic na dane uczące i dane testowe (jak w zadaniu 1b), stwórz drzewa<br/>
klasyfikacyjne na danych uczących i przetestuj na danych testowych.<br/>
<br/>
c) Spróbuj ograniczyć głębokość drzewa. Zobacz jak wpływa to na wyniki (zarówno na danych<br/>
uczących jak i na danych testowych).<br/>
<br/>
d) Spróbuj przyciąć drzewo techniką omawianą na wykładzie lub własną. Zobacz jak wpływa to na<br/>
wyniki (zarówno na danych uczących jak i na danych testowych).<br/>
<br/>
e) Zrób cross validation wszystkich tworzonych w tym zadaniu klasyfikatorów na zestawie danych<br/>
Titanic. <br/>



# a) b)


In [48]:
titanic_train = pd.read_csv('data/train.csv')
titanic_test = pd.read_csv('data/test.csv')

X = titanic_train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis = 1)
X = X.dropna()
X['Sex'] = pd.Categorical(X['Sex']).codes
X['Embarked'] = pd.Categorical(X['Embarked']).codes

X.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [177]:
def cross_validation(X, no=10, criterion='gini', max_depth=1000, min_samples_split=None, min_samples_leaf=None):
    err_sum = 0
    Y = X['Survived']
    X = X.drop('Survived', axis=1)
    indices = np.random.uniform(0, no, len(X)).astype(int)
    print('--------------------------------------')
    print(f'Cross validation using {criterion}')
    for i in range(no):
        x_train = X[indices != i]
        y_train = Y[indices != i]
        x_test = X[indices == i]
        y_test = Y[indices == i]
        
        if min_samples_split and min_samples_leaf:
            model = tree.DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, 
                                        min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
        else:
            model = tree.DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
            
        model.fit(x_train, y_train)
        predictions = model.predict(x_test)
        
        result = list(predictions == y_test)
        result_cnt = result.count(True) / len(result) * 100
          
        err_sum += result_cnt
        print(f'Test: {i} -> score: {result_cnt}\n')
        
    print(f'Final error: {err_sum / no}\n')
        
        
def build_Tree(X_train, X_test, criterion='gini', max_depth=1000, min_samples_split=None, min_samples_leaf=None):
    Y_train = X_train['Survived']
    X_train = X_train.drop('Survived', axis=1)
    Y_test = X_test['Survived']
    X_test = X_test.drop('Survived', axis=1)

    model = None
    name = 'titanic.dot'
    if min_samples_split and min_samples_leaf:
        model = tree.DecisionTreeClassifier(criterion=criterion, max_depth=max_depth, 
                                        min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)
        name = 'prunned_' + name
    else:
        model = tree.DecisionTreeClassifier(criterion=criterion, max_depth=max_depth)
        
    model.fit(X_train, Y_train)
    print(f'Classification using {criterion}')
    print(metrics.confusion_matrix(model.predict(X_test), Y_test))
    print(f'Score: {model.score(X_test, Y_test) * 100:.2f}%\n')
    
    with open(name, "w") as f:
        tree.export_graphviz(model, out_file=f, feature_names=X_train.columns)

In [178]:
X_train, X_test = train_test_split(X, test_size=0.3333)
build_Tree(X_train, X_test, criterion='gini')
build_Tree(X_train, X_test, criterion='entropy')

cross_validation(X, criterion='entropy')
cross_validation(X, criterion='gini')

Classification using gini
[[110  29]
 [ 37  62]]
Score: 72.27%

Classification using entropy
[[114  35]
 [ 33  56]]
Score: 71.43%

--------------------------------------
Cross validation using entropy
Test: 0 -> score: 74.07407407407408

Test: 1 -> score: 81.57894736842105

Test: 2 -> score: 83.56164383561644

Test: 3 -> score: 75.60975609756098

Test: 4 -> score: 72.3076923076923

Test: 5 -> score: 81.08108108108108

Test: 6 -> score: 74.50980392156863

Test: 7 -> score: 69.73684210526315

Test: 8 -> score: 81.25

Test: 9 -> score: 70.0

Final error: 76.37098407912778

--------------------------------------
Cross validation using gini
Test: 0 -> score: 69.11764705882352

Test: 1 -> score: 71.42857142857143

Test: 2 -> score: 77.94117647058823

Test: 3 -> score: 70.51282051282051

Test: 4 -> score: 83.07692307692308

Test: 5 -> score: 71.95121951219512

Test: 6 -> score: 78.37837837837837

Test: 7 -> score: 71.23287671232876

Test: 8 -> score: 82.53968253968253

Test: 9 -> score: 79.68

# c)

In [179]:
# max depth limit
build_Tree(X_train, X_test, criterion='gini', max_depth=5)
build_Tree(X_train, X_test, criterion='entropy', max_depth=5)

cross_validation(X, criterion='entropy', max_depth=5)
cross_validation(X, criterion='gini', max_depth=5)

Classification using gini
[[133  35]
 [ 14  56]]
Score: 79.41%

Classification using entropy
[[126  33]
 [ 21  58]]
Score: 77.31%

--------------------------------------
Cross validation using entropy
Test: 0 -> score: 75.92592592592592

Test: 1 -> score: 79.1044776119403

Test: 2 -> score: 74.13793103448276

Test: 3 -> score: 78.87323943661971

Test: 4 -> score: 83.13253012048193

Test: 5 -> score: 80.95238095238095

Test: 6 -> score: 80.26315789473685

Test: 7 -> score: 73.84615384615385

Test: 8 -> score: 86.41975308641975

Test: 9 -> score: 82.1917808219178

Final error: 79.48473307310599

--------------------------------------
Cross validation using gini
Test: 0 -> score: 73.4375

Test: 1 -> score: 83.33333333333334

Test: 2 -> score: 84.05797101449275

Test: 3 -> score: 80.51948051948052

Test: 4 -> score: 83.07692307692308

Test: 5 -> score: 78.57142857142857

Test: 6 -> score: 78.26086956521739

Test: 7 -> score: 79.74683544303798

Test: 8 -> score: 83.33333333333334

Test: 9 -

# d)

In [180]:
# prunning
build_Tree(X_train, X_test, criterion='gini', max_depth=5, min_samples_split=10, min_samples_leaf=5)
build_Tree(X_train, X_test, criterion='entropy', max_depth=5, min_samples_split=10, min_samples_leaf=5)

cross_validation(X, criterion='entropy', max_depth=5, min_samples_split=10, min_samples_leaf=5)
cross_validation(X, criterion='gini', max_depth=5, min_samples_split=10, min_samples_leaf=5)

Classification using gini
[[130  36]
 [ 17  55]]
Score: 77.73%

Classification using entropy
[[125  33]
 [ 22  58]]
Score: 76.89%

--------------------------------------
Cross validation using entropy
Test: 0 -> score: 77.46478873239437

Test: 1 -> score: 79.51807228915662

Test: 2 -> score: 76.81159420289855

Test: 3 -> score: 76.31578947368422

Test: 4 -> score: 84.72222222222221

Test: 5 -> score: 87.93103448275862

Test: 6 -> score: 84.93150684931507

Test: 7 -> score: 78.37837837837837

Test: 8 -> score: 81.25

Test: 9 -> score: 80.35714285714286

Final error: 80.76805294879509

--------------------------------------
Cross validation using gini
Test: 0 -> score: 84.05797101449275

Test: 1 -> score: 85.91549295774648

Test: 2 -> score: 78.68852459016394

Test: 3 -> score: 77.63157894736842

Test: 4 -> score: 74.02597402597402

Test: 5 -> score: 82.08955223880598

Test: 6 -> score: 76.38888888888889

Test: 7 -> score: 81.3953488372093

Test: 8 -> score: 80.88235294117648

Test: 9 ->

# Zadanie 3
**Random Forest i Extremely Randomized Trees**

In [181]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier

In [200]:
# TITANIC
X_titanic_train, X_titanic_test = train_test_split(X, test_size=0.3333)
Y_titanic_train = X_titanic_train['Survived']
X_titanic_train = X_titanic_train.drop('Survived', axis=1)

Y_titanic_test = X_titanic_test['Survived']
X_titanic_test = X_titanic_test.drop('Survived', axis=1)

In [201]:
random_forest = RandomForestClassifier(n_estimators=20)
extra_random_forest = ExtraTreesClassifier(n_estimators=20)

random_forest.fit(X_titanic_train, Y_titanic_train)
extra_random_forest.fit(X_titanic_train, Y_titanic_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [202]:
print('Random Forest:')
print(metrics.confusion_matrix(random_forest.predict(X_titanic_test), Y_titanic_test))
print(f'Score: {random_forest.score(X_titanic_test, Y_titanic_test) * 100:.2f}%\n')

Random Forest:
[[119  30]
 [ 23  66]]
Score: 77.73%



In [203]:
print('Extremely Randomized Trees:')
print(metrics.confusion_matrix(extra_random_forest.predict(X_titanic_test), Y_titanic_test))
print(f'Score: {extra_random_forest.score(X_titanic_test, Y_titanic_test) * 100:.2f}%\n')

Extremely Randomized Trees:
[[123  33]
 [ 19  63]]
Score: 78.15%



In [207]:
# Iris
iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['species'] = pd.Categorical.from_codes(iris.target, iris.target_names)
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [209]:
X_iris_train, X_iris_test = train_test_split(iris_df, test_size=0.3333)
Y_iris_train = X_iris_train['species']
X_iris_train = X_iris_train.drop('species', axis=1)

Y_iris_test = X_iris_test['species']
X_iris_test = X_iris_test.drop('species', axis=1)

In [210]:
random_forest = RandomForestClassifier(n_estimators=20)
extra_random_forest = ExtraTreesClassifier(n_estimators=20)

random_forest.fit(X_iris_train, Y_iris_train)
extra_random_forest.fit(X_iris_train, Y_iris_train)

ExtraTreesClassifier(bootstrap=False, class_weight=None, criterion='gini',
                     max_depth=None, max_features='auto', max_leaf_nodes=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=None,
                     oob_score=False, random_state=None, verbose=0,
                     warm_start=False)

In [213]:
print('Random Forest:')
print(metrics.confusion_matrix(random_forest.predict(X_iris_test), Y_iris_test))
print(f'Score: {random_forest.score(X_iris_test, Y_iris_test) * 100:.2f}%\n')

Random Forest:
[[14  0  0]
 [ 0 19  1]
 [ 0  2 14]]
Score: 94.00%



In [215]:
print('Extremely Randomized Trees:')
print(metrics.confusion_matrix(extra_random_forest.predict(X_iris_test), Y_iris_test))
print(f'Score: {extra_random_forest.score(X_iris_test, Y_iris_test) * 100:.2f}%\n')

Extremely Randomized Trees:
[[14  0  0]
 [ 0 19  0]
 [ 0  2 15]]
Score: 96.00%

